In [1]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

import os
# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)

## EEG data:

In [2]:
## data loading
data = mat73.loadmat('x_source.mat')  
id = scipy.io.loadmat('id.mat')
IDs = pd.DataFrame(id["id"])


In [3]:
# flattening
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2 = np.array(df2)
df2 = df2.reshape((2041,68,391)) 
df2.shape

(2041, 68, 391)

In [4]:
dupa = np.concatenate([np.expand_dims(df2[:,:,i:i+10].mean(axis = 2), axis = 2) for i in range(0,381,10)], axis = 2)
#sanity check
df2[0,0,10:20].mean()
dupa[0,0,1]
dupa.shape

(2041, 68, 39)

In [5]:
#flattening and adding id
df2 = pd.DataFrame(dupa.reshape((dupa.shape[0], -1)))
df2.shape

df2['IDs'] = IDs
df2.shape

(2041, 2653)

In [6]:
#df2['IDs']#array of arrays
#df2['IDs'][0][0] #this is what i want for 1 guy
df2['IDs'] = [df2['IDs'][i][0] for i in range(2041)] #removing these weird square brackets frm ids
df2.shape

(2041, 2653)

## Behavioral Data:


In [7]:
behaviour_data = pd.read_csv('scores.csv')

### We need columns with:
1. IDs

 
2. for Anxiety:  "SCARED_SR" "SCARED_P". 
    


2. for ADHD inattention: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN IN")


3. for ADHD combined: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN HY")


4. for Autism Social Responsiveness Scale-2 ("SRS_Pre" (preschool) or "SRS") 


5. for Depression: Children's Depression Index (average score of: "CDI_SR" "CDI_P") (alternative 1: Mood & Feelings Questionnaire (MFQ); Alternative 2: Affective Reactivity Index—(ARI-S) Self Report)


6. WISC-Full IQ "WISC_FSIQ"


In [8]:
behaviour_data = behaviour_data[['IDs', 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score', 'WISC_FSIQ' ]]
behaviour_data.shape

(3076, 8)

In [9]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 8 columns (with column 'IDs').
There are 7 columns that have missing values.


,Missing Values,% of Total Values
WISC_FSIQ,588,19.1
MFQ_P_Total,466,15.1
SCARED_P_Total,328,10.7
SRS_Total,218,7.1
SWAN_HY,163,5.3
SWAN_IN,163,5.3
ARI_S_Total_Score,125,4.1


## Merge EEG and behavioral data together:

In [10]:
df = pd.merge(df2, behaviour_data, on='IDs', how='inner')
df.shape

(1929, 2660)

## features and labels preparation:

In [11]:
#removing NaNs
df = df.dropna()
df.shape

(1465, 2660)

In [12]:

X = df.iloc[:, 0:2652].to_numpy()



In [13]:
Y = df[[ 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score', 'WISC_FSIQ' ]]

#### Data normalisation:

In [14]:
Y = Y.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
Y_scaled = min_max_scaler.fit_transform(Y)
Y = pd.DataFrame(Y_scaled)
Y.shape

(1465, 7)

#### PCA on X dimension:


In [47]:
# dimensionality reduction
#pca = PCA(.95) # 95% variance retained
#pca.fit(X)

# transform data
#X_pca = pca.transform(X)
#X_pca.shape
#X_pca = X

In [15]:
print("X:", X.shape, "Y:", Y.shape)

X: (1465, 2652) Y: (1465, 7)


In [16]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.15)

## MODELLING PART

In [17]:
max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=10, max_depth=max_depth, random_state=0, verbose =1)
)
regr_multirf.fit(xtrain, ytrain)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

keras, simple MLP

In [18]:
model = Sequential()
model.add(Dense(100, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", metrics = ['mse'], optimizer="adam")
model.fit(xtrain, ytrain, epochs=100, batch_size=12, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)


#ypred

2021-11-17 20:30:32.807159: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-17 20:30:32.840340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-17 20:30:32.841006: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-11-17 20:30:32.841223: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-17 20:30:32.842357: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-17 20:30:32.843641: I tensorflow/stream_executor/platform/default/d

Epoch 1/100
 1/83 [..............................] - ETA: 0s - loss: 0.8225 - mse: 0.8225

2021-11-17 20:30:33.743238: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


83/83 [==============================] - 0s 3ms/step - loss: 0.1004 - mse: 0.1004 - val_loss: 0.0501 - val_mse: 0.0501
Epoch 2/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0455 - mse: 0.0455 - val_loss: 0.0543 - val_mse: 0.0543
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0472 - mse: 0.0472 - val_loss: 0.0480 - val_mse: 0.0480
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0443 - mse: 0.0443 - val_loss: 0.0471 - val_mse: 0.0471
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0380 - mse: 0.0380 - val_loss: 0.0472 - val_mse: 0.0472
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0383 - mse: 0.0383 - val_loss: 0.0480 - val_mse: 0.0480
Epoch 7/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0350 - mse: 0.0350 - val_loss: 0.0474 - val_mse: 0.0474
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0355 - mse: 0.0355 -

In [19]:
dupa2 = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)

dupa2 = np.array(dupa2)

dupa2.mean()

0.06788103

In [ ]:
#xtrain = np.asarray(xtrain).astype('float32')
#xtest = np.asarray(xtest).astype('float32')

In [20]:
from tensorflow.keras.layers import Conv2D , MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
#dilation of the filters , for deeper  convolutionm try with the whole signal
def get_CNN_2_classifier():

    k = 3  # kernel size
    s = 2  # stride
    n_filters = 32 # number of filters
    inputs = Input(shape=(39, 68, 1))

    conv1 = Conv2D(n_filters, kernel_size=(k, k), strides=(s, s), padding='same')(inputs)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = Conv2D(n_filters, kernel_size=(k, k), padding='same')(conv1)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
    pool1 = conv1 # MaxPooling2D(pool_size=(s, s))(conv1)

    conv2 = Conv2D(2 * n_filters, kernel_size=(k, k), strides=(s, s), padding='same')(pool1)
    conv2 = BatchNormalization(scale=False, axis=3)(conv2)
    conv2 = LeakyReLU(alpha=0.3)(conv2)
    conv2 = Conv2D(2 * n_filters, kernel_size=(k, k), padding='same')(conv2)
    conv2 = BatchNormalization(scale=False, axis=3)(conv2)
    conv2 = LeakyReLU(alpha=0.1)(conv2)
    pool2 = MaxPooling2D(pool_size=(s, s))(conv2)

    conv3 = Conv2D(4 * n_filters, kernel_size=(k, k), padding='same')(pool2)
    conv3 = BatchNormalization(scale=False, axis=3)(conv3)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    conv3 = Conv2D(4 * n_filters, kernel_size=(k, k), padding='same')(conv3)
    conv3 = BatchNormalization(scale=False, axis=3)(conv3)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    pool3 = MaxPooling2D(pool_size=(s, s))(conv3)

    conv4 = Conv2D(8 * n_filters, kernel_size=(k, k), padding='same')(pool3)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    conv4 = Conv2D(8 * n_filters, kernel_size=(k, k), padding='same')(conv4)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D(pool_size=(s, s))(conv4)

    conv5 = Conv2D(16 * n_filters, kernel_size=(k, k), padding='same')(pool4)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)
    conv5 = Conv2D(16 * n_filters, kernel_size=(k, k), padding='same')(conv5)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)

    gap = GlobalAveragePooling2D()(conv5)
    outputs = Dense(7, activation='relu')(gap)

    model = Model(inputs, outputs)
    model.compile(optimizer = Adam(lr=1e-4), loss = 'mse', metrics = ['mse'])
    model.summary()
  
    return model

CNN_model_2 = get_CNN_2_classifier()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 39, 68, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 34, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 34, 32)        96        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 20, 34, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 34, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 34, 32)        96        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 20, 34, 32)        0     

In [21]:
# data reshaping for CNN
n_electrode = 68
n_freq =39 

xtrain = xtrain.reshape((xtrain.shape[0], n_electrode, n_freq))
xtest = xtest.reshape((xtest.shape[0], n_electrode, n_freq))



In [22]:
history_CNN_2 = CNN_model_2.fit(np.transpose(xtrain, (0, 2, 1)), ytrain, validation_split=0.2, epochs=50, verbose=1)


ypred = CNN_model_2.predict(np.transpose(xtest, (0, 2, 1)))



dupa2 = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)

dupa2 = np.array(dupa2)

dupa2.mean()


Epoch 1/50


2021-11-17 20:31:03.699898: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


32/32 [==============================] - 1s 33ms/step - loss: 0.1572 - mse: 0.1572 - val_loss: 0.1891 - val_mse: 0.1891
Epoch 2/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0850 - mse: 0.0850 - val_loss: 0.1589 - val_mse: 0.1589
Epoch 3/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0492 - mse: 0.0492 - val_loss: 0.1291 - val_mse: 0.1291
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0335 - mse: 0.0335 - val_loss: 0.1139 - val_mse: 0.1139
Epoch 5/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.0942 - val_mse: 0.0942
Epoch 6/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0190 - mse: 0.0190 - val_loss: 0.0832 - val_mse: 0.0832
Epoch 7/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0602 - val_mse: 0.0602
Epoch 8/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0134 - mse: 0.0134 

0.04889931